In [3]:
import pandas as pd

volatility_df = pd.read_csv("data/filtered_volatility_df.csv", parse_dates=["timestamp"])
volatility_df.head()

,timestamp,price,tweet_id,handle,ticker,sentiment_label,sentiment_score,top_topic,Technology,Healthcare and Pharmaceuticals,...,Energy,Agriculture,Automotive,minutes_since_tweet,log_return,volatility,price_change_pct,average_volatility,price_spike,weighted_volatility
0,2016-12-05 12:15:00,99.002000,805384490533212160,@realDonaldTrump,WYNN,Neutral,0.949813,Technology,0.464619,0.449575,...,0.318766,0.122376,0.381047,1441.658,NaN,0.017418,NaN,0.321163,0,0.017418
1,2016-12-05 12:15:00,27.458544,805384490533212160,@realDonaldTrump,FOX,Neutral,0.949813,Technology,0.464619,0.449575,...,0.318766,0.122376,0.381047,1441.658,-1.282463,0.004898,NaN,0.058880,1,0.007542
2,2016-12-05 12:15:00,61.312195,805384490533212160,@realDonaldTrump,LVS,Neutral,0.949813,Technology,0.464619,0.449575,...,0.318766,0.122376,0.381047,1441.658,NaN,0.016306,NaN,0.125180,1,0.034060
3,2016-12-05 12:15:00,29.001725,805384490533212160,@realDonaldTrump,MGM,Neutral,0.949813,Technology,0.464619,0.449575,...,0.318766,0.122376,0.381047,1441.658,-0.748623,0.007139,NaN,0.062956,1,0.011124
4,2016-12-05 12:16:00,98.930269,805384490533212160,@realDonaldTrump,WYNN,Neutral,0.949813,Technology,0.464619,0.449575,...,0.318766,0.122376,0.381047,1442.658,1.227060,0.036447,NaN,0.321163,1,0.140217


# Apply **ARIMA** forecasting

In [4]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

In [ ]:
def arima_forecast(df, column='weighted_volatility', steps=10):

    df.reset_index(inplace=True)
    df.set_index('timestamp', inplace=True)

    df = df.loc[~df.index.duplicated(keep='first')]

    df = df.asfreq('s')  # Set frequency to 's' (seconds)

    series = df[column].dropna()

    #fit ARIMA model
    model = ARIMA(series, order=(1, 1, 1))
    model_fit = model.fit()

    #forecast future values
    forecast = model_fit.forecast(steps=steps)

    # Create a date range
    forecast_dates = pd.date_range(df.index[-1], periods=steps + 1, freq='s')[1:]

    return pd.Series(forecast, index=forecast_dates, name=f'{column}_ARIMA_Forecast')

In [ ]:
arima_forecasted = arima_forecast(volatility_df, column='weighted_volatility', steps=10)

# Apply **GARCH** forecasting

In [ ]:
!pip install arch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 11.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model
import matplotlib.pyplot as plt

In [ ]:
def garch_forecast(df, column='weighted_volatility', steps=5):
    if 'timestamp' not in df.columns:
        df.reset_index(inplace=True)  # Ensure timestamp column is kept
    df.set_index('timestamp', inplace=True)

    df = df.loc[~df.index.duplicated(keep='first')]
    df = df.asfreq('s')  # Set frequency to 's' (seconds)


    returns = df[column].dropna()
    returns_scaled = returns * 0.0001

    # Fit GARCH(1, 1) model
    model = arch_model(returns_scaled, vol='Garch', p=1, q=1)
    model_fit = model.fit()

    # Forecast volatility
    forecast_volatility = model_fit.forecast(horizon=steps)

    # Get forecasted variance
    forecast_variance = forecast_volatility.variance.values[-1, :]
    forecast_volatility = pd.Series(np.sqrt(forecast_variance), index=pd.date_range(df.index[-1], periods=steps + 1, freq='s')[1:], name=f'{column}_GARCH_Forecast')

    return forecast_volatility


In [ ]:
garch_forecasted = garch_forecast(volatility_df, column='weighted_volatility', steps=5)
volatility_df['GARCH_Forecast'] = garch_forecasted

volatility_df.head()